In [ ]:
import os
import cbsodata
import pandas as pd
import numpy as np

In [ ]:
def get_cbs_data(dataset, cache=True):
    cache_dir = 'cbs_cache'

    cache_file = os.path.join(cache_dir,dataset+".pickle")
    if cache and os.path.exists(cache_file):
        ds = pd.read_pickle(cache_file)
        return ds, None
    else:
        ds = pd.DataFrame(cbsodata.get_data(dataset))
        for c in ds.columns:
            if ds[c].dtype.str == '|O':
                ds[c] = ds[c].str.strip()
        ds.to_pickle(cache_file)

    cache_file2 = os.path.join(cache_dir,dataset+"_info.pickle")
    if cache and os.path.exists(cache_file2):
        columninfo = pd.read_pickle(cache_file2)
    else:
        columninfo = pd.DataFrame(cbsodata.get_meta(dataset, 'DataProperties'))
        columninfo.to_pickle(cache_file2)
    if type(ds) == tuple :
        ds = ds[0]
    return ds, columninfo

In [ ]:
# Obtain Brent oil barrel price data
# Source: https://www.investing.com/commodities/brent-oil
# Historical data can be downloaded with a free account

# Since oil price is in dollars and we are looking at long term effects,
# the price must be converted to euros. The euro-dollar  trading price
# is retrieved from the same source: https://www.investing.com/currencies/eur-usd

oildata = pd.merge(pd.read_csv('Brent Oil Futures Historical Data Daily.csv'),
                   pd.read_csv('EUR_USD Historical Data Daily.csv').add_suffix('_USD'), 
                   left_on='Date', right_on='Date_USD')
oildata['Price_EUR'] = oildata['Price']*oildata['Price_USD']
import locale
locale.setlocale(locale.LC_ALL, "uk_UK")
oildata["Date"] = pd.to_datetime(oildata["Date"], format='%b %d, %Y')
oildata = oildata[['Date', 'Price_EUR']].rename(columns={'Price_EUR': 'Oil'})
oildata.head(10)

In [ ]:
# Retrieve the price for E10 and Diesel fuels at Dutch pumps
# This data is avalailable at CBS, in dataset 80416NED
# A wrapper is used around the cbsodata package to enable caching

gasdataNL , _ = get_cbs_data("80416NED")
import locale
locale.setlocale(locale.LC_ALL, "nl_NL")
gasdataNL['Perioden'] = pd.to_datetime(gasdataNL['Perioden'], format='%Y %A %d %B')
gasdataNL = gasdataNL.rename(columns={'BenzineEuro95_1' : 'E10_NL', 'Diesel_2': 'Diesel_NL','Perioden': 'Date'})
gasdataNL = gasdataNL.drop(columns=['ID', 'Lpg_3'])
gasdataNL.head(2)

In [ ]:
# Retrieve the fule prices from Germany.
# Source Diesel: https://www.finanzen.net/rohstoffe/diesel-benzinpreis/historisch
# Source Benzin: https://www.finanzen.net/rohstoffe/super-benzinpreis/historisch

# Use F12 and the webdev tools to select the table on the webpage
# Copy the internal HTML of this table and paste in Excel (Excel understands HTML table structures)
# Clean up in Excel and save as CSV
# The automated way is using BeautifoulSoup, but for this tutorial we use the steps above

gasdataD = pd.merge(
           pd.read_csv('Deutschland_Superbenzin.csv', sep=';').rename(columns={'Datum': 'Date', "Schluss": "E10_D"}),
           pd.read_csv('Deutschland_Diesel.csv', sep=';').rename(columns={'Datum': 'Date', "Schluss": "Diesel_D"}),
           how='outer'
       )
gasdataD['Date'] = pd.to_datetime(gasdataD['Date'], format='%d.%m.%Y')
gasdataD['E10_D'] = gasdataD['E10_D'].str.replace(',', '.').astype(float)
gasdataD['Diesel_D'] = gasdataD['Diesel_D'].str.replace(',', '.').astype(float)
gasdataD.head(2)

In [ ]:
# Combine the fule prices to one table
gasdata = pd.merge(gasdataNL, gasdataD, how='outer').sort_values('Date')
gasdata.tail(2)

In [ ]:
# Combine fuel prices with the oil price
# Resample data so we have a row for each day, fill missing data
# Make a rolling average of 7 days to smooth out the data
df_gas = pd.merge(oildata, gasdata, left_on='Date', right_on='Date', how='outer').set_index('Date')
df_gas = df_gas.resample('1d').max().interpolate().rolling(7).mean()
df_gas = df_gas[df_gas.index >= '2020-01-01']
df_gas.head(2)

In [ ]:
# Create some additional datafields
df_gas['Oil_rel']=df_gas['Oil']/df_gas.iloc[0]['Oil']

df_gas['E10_NL_2_Oil'] = df_gas['E10_NL']/df_gas['Oil']
df_gas['E10_NL_rel']=df_gas['E10_NL']/df_gas.iloc[0]['E10_NL']
df_gas['Diesel_NL_rel']=df_gas['Diesel_NL']/df_gas.iloc[0]['Diesel_NL']

df_gas['E10_D_2_Oil'] = df_gas['E10_D']/df_gas['Oil']
df_gas['E10_D_rel']=df_gas['E10_D']/df_gas.iloc[0]['E10_D']
df_gas['Diesel_D_rel']=df_gas['Diesel_D']/df_gas.iloc[0]['Diesel_D']

df_gas['E10_NL_2_D'] = df_gas['E10_NL']/df_gas['E10_D']
df_gas['E10_NL_2_D_diff'] = df_gas['E10_NL']-df_gas['E10_D']
df_gas['E10_NL_2_D_diff_rel'] = (df_gas['E10_NL']-df_gas['E10_D'])/df_gas['E10_NL']

In [ ]:
# Plot the oil price and the fuel price of E10 (NL and D)
ax = df_gas['Oil'].plot(legend=True, figsize=(15,5))
ax.set_ylabel('Oil price')
# ax = df_gas[['E10_NL', 'E10_D']].plot(secondary_y=True, ax=ax, legend=True)
ax = df_gas[['E10_NL']].plot(secondary_y=True, ax=ax, legend=True)
ax.set_title('Oil and E10 prices over time')
_ = ax.set_ylabel('E10 price')

In [ ]:
# Plot the difference between the Dutch and German prices
ax = df_gas['E10_NL_2_D_diff'].plot(figsize=(15,5))
ax.set_title('Price difference in gasoline between NL and D')
ax.set_ylabel('Price difference (euro)')

In [ ]:
# Try to build a predictor

In [ ]:
# Combine oil and fuel prices and create some additional features (historical data shift)
df_gas = pd.merge(gasdata, oildata, left_on='Date', right_on='Date').set_index('Date').interpolate().dropna()
df_gas = df_gas.resample('1d').max().interpolate().rolling(7, min_periods=1).mean().sort_values('Date')
df_gas = df_gas[df_gas.index > '2021-01-01']

df_gas['Oil_7d'] = df_gas['Oil'].shift(7)
df_gas['Oil_14d'] = df_gas['Oil'].shift(14)
df_gas['Oil_delta'] = df_gas['Oil_14d'] - df_gas['Oil_7d']

df_gas['E10_NL_7d'] = df_gas['E10_NL'].shift(7)
df_gas['E10_NL_14d'] = df_gas['E10_NL'].shift(14)
df_gas['E10_NL_delta'] = df_gas['E10_NL_14d'] - df_gas['E10_NL_7d']

In [ ]:
# Compare Fuel price with oil price(same date, 7 days back 14 days back)
df_plot = df_gas[df_gas.index > '2022-01-01']
ax = df_plot[['Oil']].plot(legend=True, figsize=(15,5))
ax.legend()
df_plot[['Oil_7d', 'Oil_14d']].plot(ax=ax, legend=True, ls='--')
df_plot[['E10_NL']].plot(secondary_y=True, ax=ax, legend=True, lw=2)
ax.legend()

In [ ]:
# Show correlations
df_gas.corr()

In [ ]:
df = df_gas.dropna().copy()
# define the predictors  
predictors = ['Oil', 'Oil_7d', 'Oil_14d', 'E10_NL_7d', 'E10_NL_14d']
# define the target column
target = 'E10_NL'

# Do not use the last 7 days for training the model
# Note the difference in argument order
dropN = 7
X = df[predictors][:-dropN]
y = df[[target]][:-dropN]

In [ ]:
from sklearn import svm
regr = svm.SVR()
regr.fit(X, y)
df['EstSVM'] = regr.predict(df[predictors])
#regr.predict(df[predictors])

In [ ]:
import statsmodels.api as sm
model = sm.OLS(y, X).fit()
# Score the model
df['EstLR'] = model.predict(df[predictors])
#model.predict(df[predictors])

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the dataset in train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# Scale inputs
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X_train)
X_testscaled=sc_X.transform(X_test)

reg = MLPRegressor(hidden_layer_sizes=(64,64,64),activation="relu" ,random_state=1, max_iter=5000)
reg.fit(X_trainscaled, y_train)

# scale inputs before predicting
df['EstMLP'] = reg.predict(sc_X.fit_transform(df[predictors]))

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
print("                                 R2-score RMSE     MAE")
print("            The Score with LR  : {:5.4f}   {:6.5f}  {:6.5f}".format((r2_score(df['EstLR'], df['E10_NL'])),
                                                   mean_squared_error(df['EstLR'], df['E10_NL']),
                                                   mean_absolute_error(df['EstLR'], df['E10_NL'])))
print("            The Score with MLP : {:5.4f}   {:6.5f}  {:6.5f}".format((r2_score(df['EstMLP'], df['E10_NL'])),
                                                   mean_squared_error(df['EstMLP'], df['E10_NL']),
                                                   mean_absolute_error(df['EstMLP'], df['E10_NL'])))
print("            The Score with SVM : {:5.4f}   {:6.5f}  {:6.5f}".format((r2_score(df['EstSVM'], df['E10_NL'])),
                                                   mean_squared_error(df['EstSVM'], df['E10_NL']),
                                                   mean_absolute_error(df['EstSVM'], df['E10_NL'])))


In [ ]:
# Filter last records that were not part of the model input
df['E10_NL_input'] = df['E10_NL']
for i in range(-dropN,0,1):
    df.iloc[i, df.columns.get_loc('E10_NL_input')] = None
df_plot = df[['E10_NL_input', 'E10_NL', 'EstLR', 'EstSVM', 'EstMLP']][df.index > '2022-01-01']

ax = df_plot[['E10_NL_input']].plot(color='blue', ls='-', figsize=(15,5))
ax = df_plot[['E10_NL']].plot(ax=ax, color='blue', ls='--')
ax = df_plot[['EstLR']].plot(ax=ax, color='green')
ax = df_plot[['EstSVM']].plot(ax=ax, color='red')
ax = df_plot[['EstMLP']].plot(ax=ax, color='orange')